# Importar librerias 

In [20]:
import janitor
import matplotlib.pyplot as plt
import missingno
import numpy as np
import pandas as pd
import pyreadr
import seaborn as sns
import session_info
import upsetplot

# Configurar aspecto de las graficas

In [21]:
#para que los graficos sean abiertos directamente en el notebook y no en ventana emergente
%matplotlib inline 

#Estilo a las graficas con la biblioteca seaborn
sns.set(
    rc={
        "figure.figsize": (10, 10)
    }
)

sns.set_style("whitegrid")

# Importar funciones personalizadas

In [39]:
%run pandas-missing-extension.ipynb

# Operador con valores faltantes

## Python

In [22]:
print(
    None or True,
    None or False,
    None == None,
    None is None,
    # None + True -> esta linea generar error para Python al trabajar con datos None
    type(None),
    sep="\n"
)

True
False
True
True
<class 'NoneType'>


## Numpy

In [23]:
print(
    np.nan or True,
    np.nan == np.nan,
    np.nan is np.nan,
    np.nan + 2, # En esta libreria si puedo hacer calculos con valores faltantes
    f"los valores faltantes con Numpy son de tipo: {type(np.nan)}",
    np.isnan(np.nan),
    sep="\n"

)

nan
False
True
nan
los valores faltantes con Numpy son de tipo: <class 'float'>
True


## Pandas

In [24]:
test_missing_df = pd.DataFrame.from_dict(
    data=dict(
        x=[0, 1, np.nan, np.nan, None],
        y=[0, 1, pd.NA, np.nan, None]
    )
)

test_missing_df

#En la tabla a continuación nos damos cuenta como muestra lso datos faltantes Pandas <NA>

,x,y
0,0.0,0
1,1.0,1
2,NaN,<NA>
3,NaN,NaN
4,NaN,None


In [25]:
# Para saber existencia de datos faltantes, y que sin importar como se representen el nos mostrara True or False
test_missing_df.isnull()

,x,y
0,False,False
1,False,False
2,True,True
3,True,True
4,True,True


In [26]:
# Para saber existencia de datos faltantes, y que sin importar como se representen el nos mostrara True or False
test_missing_df.isnull()

,x,y
0,False,False
1,False,False
2,True,True
3,True,True
4,True,True


In [27]:
# Para saber datos faltantes en una columna 
test_missing_df.x.isnull()

0    False
1    False
2     True
3     True
4     True
Name: x, dtype: bool

In [28]:
# Pandas nos refleja valores faltantes cunado se mezclan con float, como NaN
pd.Series([1,np.nan])

0    1.0
1    NaN
dtype: float64

In [29]:
#Pandas nos entrega valores faltantes cuando se mezclan con fechas, como NaT
pd.Series([pd.to_datetime("2022-01-01"), np.nan])


0   2022-01-01
1          NaT
dtype: datetime64[ns]

<div class="alert alert-info" role="alert">
    <b style="font-size: 1.5em;">🔍 Summary</b>
    <p>
        Después de analizar los datos y realizar diversas exploraciones, llegamos a las siguientes conclusiones:
        <ul>
            <li>Es crucial identificar los valores faltantes, dependiendo de la biblioteca que estés utilizando.</li>
            <li>En Numpy, los valores faltantes se representan como 'nan' y se comportan como valores numéricos (tipo float). Al compararlos con 'None', se obtiene False, indicando que no son el mismo objeto. Sin embargo, al usar la función 'is', se obtiene True, lo que significa que un 'nan' está contenido en otro 'nan'. Se debe tener precaución al manejar estos casos.</li>
        </ul>
    </p>
</div>

# Carga los conjuntos de datos

## Prima Indias Diabetes

In [30]:
prima_indians_diabetes_url = "https://nrvis.com/data/mldata/pima-indians-diabetes.csv"

In [31]:
# Se usa el comando " ! " para escribir comandos que escribirias en la terminal
!wget -O ../data/prima-indians-diabetes.csv { prima_indians_diabetes_url } -q

In [32]:
diabetes_df = pd.read_csv(
    "../data/prima-indians-diabetes.csv", # or prima_indisas_diabetes_url
    sep=",",
    #El dataset venia sin nombrar las columnas por tal motivo es necesario hacerlo manual
names = [
    "pregnancies",  # embarazos
    "glucose",  # glucosa
    "blood_pressure",  # presion_sanguinea
    "skin_thickness",  # espesor_piel
    "insulin",  # insulina
    "bmi",  # bmi
    "diabetes_pedigree_function",  # funcion_pedigree_diabetes
    "age",  # edad
    "outcome",  # resultado
]
)

## naniar (oceanbuoys, pedestrian, riskfactors)

## Crear unidades de información de los conjuntos de datos

In [33]:
base_url = "https://github.com/njtierney/naniar/raw/master/data/"
datasets_names = ("oceanbuoys", "pedestrian", "riskfactors")
extension = ".rda"

### Descargar y cargar los conjuntos de datos

In [34]:
# Diccionario para almacenar los DataFrames de los conjuntos de datos
datasets_dfs = {}

# Iterar sobre los nombres de los conjuntos de datos
for dataset_name in datasets_names:

    # Construir nombres de archivo y URL para el conjunto de datos actual
    dataset_file = f"{ dataset_name }{ extension }"
    dataset_output_file = f"../data/{ dataset_file }"
    dataset_url = f"{ base_url }{ dataset_file }"
    
    # Descargar el conjunto de datos desde la URL y guardar en el directorio ../data/
    !wget -q -O { dataset_output_file } { dataset_url }

    # Leer el conjunto de datos desde el archivo usando pyreadr y almacenarlo en el diccionario
    datasets_dfs[f"{ dataset_name }_df"] = pyreadr.read_r(dataset_output_file).get(dataset_name)

# Mostrar las claves (nombres) de los DataFrames almacenados en el diccionario
datasets_dfs.keys()

dict_keys(['oceanbuoys_df', 'pedestrian_df', 'riskfactors_df'])

### incluir conjunto de datos en nuestro ambiente local

In [35]:
locals().update(**datasets_dfs)
del datasets_dfs

### Verificar carga

In [36]:
oceanbuoys_df.shape, pedestrian_df.shape, riskfactors_df.shape, diabetes_df.shape

((736, 8), (37700, 9), (245, 34), (768, 9))

In [37]:
riskfactors_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   state             245 non-null    category
 1   sex               245 non-null    category
 2   age               245 non-null    int32   
 3   weight_lbs        235 non-null    object  
 4   height_inch       243 non-null    object  
 5   bmi               234 non-null    float64 
 6   marital           244 non-null    category
 7   pregnant          30 non-null     category
 8   children          245 non-null    int32   
 9   education         244 non-null    category
 10  employment        245 non-null    category
 11  income            245 non-null    category
 12  veteran           242 non-null    category
 13  hispanic          243 non-null    category
 14  health_general    245 non-null    category
 15  health_physical   245 non-null    int32   
 16  health_mental     245 non-

# Examples markdown


In [38]:
# Experimento para cuadno temrine
%run download_data_00.ipynb
#diabetes_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 245 entries, 0 to 244
Data columns (total 34 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   state             245 non-null    category
 1   sex               245 non-null    category
 2   age               245 non-null    int32   
 3   weight_lbs        235 non-null    object  
 4   height_inch       243 non-null    object  
 5   bmi               234 non-null    float64 
 6   marital           244 non-null    category
 7   pregnant          30 non-null     category
 8   children          245 non-null    int32   
 9   education         244 non-null    category
 10  employment        245 non-null    category
 11  income            245 non-null    category
 12  veteran           242 non-null    category
 13  hispanic          243 non-null    category
 14  health_general    245 non-null    category
 15  health_physical   245 non-null    int32   
 16  health_mental     245 non-

<div class="alert alert-info" role="alert">
    <b style="font-size: 1.5em;">🔍 Summary</b>
    <p>
        Después de analizar los datos y realizar diversas exploraciones, llegamos a las siguientes conclusiones:
        <ul>
            <li>Es crucial identificar los valores faltantes, dependiendo de la biblioteca que estés utilizando.</li>
            <li>En Numpy, los valores faltantes se representan como 'nan' y se comportan como valores numéricos (tipo float). Al compararlos con 'None', se obtiene False, indicando que no son el mismo objeto. Sin embargo, al usar la función 'is', se obtiene True, lo que significa que un 'nan' está contenido en otro 'nan'. Se debe tener precaución al manejar estos casos.</li>
        </ul>
    </p>
</div>

<div class="alert alert-info" role="alert">
    <b style="font-size: 1.5em;">ℹ️ Información</b>
    <p>
        Este es un mensaje informativo.
    </p>
</div>

<div class="alert alert-success" role="alert">
    <b style="font-size: 1.5em;">✅ Éxito</b>
    <p>
        ¡La operación se completó con éxito!
    </p>
</div>